# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation = pd.read_csv("../Weatherpy/output_data/cities.csv", encoding="utf-8")
humidity = vacation["Humidity"].astype(float)
vacation.head()


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,Avarua,-21.2078,-159.7750,82.45,78,20,13.80,1644183575
1,1,Pundaguitan,6.3711,126.1689,80.13,77,86,17.74,1644183576
2,2,Tuzla,44.5428,18.6694,35.56,93,0,2.30,1644183576
3,3,Tuktoyaktuk,69.4541,-133.0374,-25.60,75,75,18.41,1644183540
4,4,Saskylakh,71.9167,114.0833,-23.28,100,96,2.37,1644183576


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
latlong = vacation[["Latitude", "Longitude"]].astype(float)
heatmap = gmaps.figure()
maxhum = vacation['Humidity'].max()
layer = gmaps.heatmap_layer(latlong, weights=humidity, 
                                 dissipating=False, max_intensity=maxhum,
                                 point_radius=2)

heatmap.add_layer(layer)

heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
#temp btwn 80 and 70
spot = vacation.loc[(vacation["Max Temp"] < 80) & (vacation["Max Temp"] > 70)]

# Wind speed less than 10 mph.
spot = vacation.loc[vacation["Wind Speed"] < 10]

# Zero cloudiness.
spot = vacation.loc[vacation["Cloudiness"] == 0]
spot.dropna(how='any')
spot.reset_index(inplace=True)
del spot['index']
spot.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,2,Tuzla,44.5428,18.6694,35.56,93,0,2.30,1644183576
1,20,Anadyr,64.7500,177.4833,-19.52,75,0,6.71,1644183579
2,21,Provideniya,64.3833,-173.3000,1.51,60,0,2.24,1644183579
3,23,Lebu,-37.6167,-73.6500,64.06,74,0,13.89,1644183580
4,29,Saint Paul Harbor,57.7900,-152.4072,36.91,65,0,14.97,1644183581


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = spot

In [20]:
hotels = []

for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['Latitude']
    lng = hotel_df.loc[i]['Longitude']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotels.append(jsn['results'][0]['name'])
    except:
        hotels.append("")
hotel_df["Hotel Name"] = hotels
hotel_df = hotel_df.dropna(how='any')
hotel_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
0,2,Tuzla,44.5428,18.6694,35.56,93,0,2.30,1644183576,Tuzla
1,20,Anadyr,64.7500,177.4833,-19.52,75,0,6.71,1644183579,Anadyr
2,21,Provideniya,64.3833,-173.3000,1.51,60,0,2.24,1644183579,Provideniya
3,23,Lebu,-37.6167,-73.6500,64.06,74,0,13.89,1644183580,Lebu
4,29,Saint Paul Harbor,57.7900,-152.4072,36.91,65,0,14.97,1644183581,Kodiak


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [24]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
heatmap.add_layer(markers)


# Display figure
heatmap

Figure(layout=FigureLayout(height='420px'))